In [1]:
from ddCRP import ddCRP
from ddCRP import Priors

from surface_utilities import adjacency

import numpy as np
import nibabel as nb
from niio import loaded

In [2]:
data_file = '/Users/kristianeschenburg/Desktop/Research/Data/CorticalRegionalization/Destrieux/RestingState/285345.L.Cortical.Regionalized.RestingState.aparc.a2009s.mat'
label_file = '/Users/kristianeschenburg/Desktop/Research/Data/Labels/HCP/285345.L.CorticalAreas.fixed.32k_fs_LR.label.gii'
surf_file = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/285345.L.inflated.32k_fs_LR.surf.gii'

In [3]:
# Load feature matrix
data = loaded.load(data_file)
[n,p] = data.shape
if n < p:
    data = data.T

In [4]:
# Get indices of non-zero data samples
indices = np.abs(data).sum(1) != 0
idx = np.arange(data.shape[0])[indices]

features = np.arange(data.shape[1])[np.var(data,axis=0) != 0]

In [5]:
# Load surface, generate adjacency list
surface = nb.load(surf_file)
vertices = surface.darrays[0].data
faces = surface.darrays[1].data

S = adjacency.SurfaceAdjacency(vertices=vertices,faces=faces)
S.generate()

# Filter adjacency list
filtered = S.filtration(S.adj,filter_indices=idx,remap=True)

In [6]:
print('Keys in adj list: {:}'.format(len(filtered.keys())))

Keys in adj list: 29696


In [7]:
# Load initial clustering
label = loaded.load(label_file)

In [8]:
# Filter data matrix and label array
filtered_data = data[idx,:][:,features]
print('Data shape: {:}'.format(filtered_data.shape))

init_z_filtered = label[idx]
print('Label shape: {:}'.format(init_z_filtered.shape))
print('Unique labels: {:}'.format(len(np.unique(init_z_filtered))))

Data shape: (29696, 74)
Label shape: (29696,)
Unique labels: 172


In [9]:
lab2order = dict(zip(np.unique(init_z_filtered),np.arange(len(init_z_filtered))))
order2lab = dict(zip(lab2order.values(),lab2order.keys()))

init_z_remap = np.zeros((init_z_filtered.shape))

for k in lab2order.keys():
    idx = np.where(init_z_filtered == k)[0]
    init_z_remap[idx] = lab2order[k]
    
print('Remapped label shape: {:}'.format(init_z_remap.shape))
print('Unique remapped labels: {:}'.format(len(np.unique(init_z_remap))))

Remapped label shape: (29696,)
Unique remapped labels: 172


In [40]:
# Set hyperparameters for prior / likelihood model
alpha = 10
mu_bar = np.zeros((filtered_data.shape[1],))
kappa = 0.001
nu = 150
lambda_bar = np.eye(len(mu_bar)) + np.random.rand(filtered_data.shape[1])

# Define prior
niw = Priors.NIW(mu0 = mu_bar, kappa0=kappa, nu0=nu, lambda0=lambda_bar)

In [41]:
import importlib
importlib.reload(ddCRP)
importlib.reload(ddCRP.subgraphs)

<module 'ddCRP.subgraphs' from '/Users/kristianeschenburg/Documents/Code/ddCRP/ddCRP/subgraphs.py'>

In [42]:
# Initialize ddCRP model
crp = ddCRP.ddCRP(alpha=alpha, mcmc_passes=300, n_clusters=180, stats_interval=5, model=niw)

In [ ]:
# Fit model
crp.fit(adj_list=filtered,features=filtered_data, init_z=init_z_remap)


Full
Step: 0 Time: -1536562227.3047569 LP: -427816.94931985706 K: 192 MaxLP: -427816.94931985706
Step: 5 Time: -1536562222.9475589 LP: -426357.8766637056 K: 192 MaxLP: -426357.8766637056
Step: 10 Time: -1536562218.639071 LP: -426357.8766637056 K: 192 MaxLP: -426357.8766637056
Step: 15 Time: -1536562214.361774 LP: -426322.5029144129 K: 192 MaxLP: -426322.5029144129
Step: 20 Time: -1536562210.051107 LP: -426322.5029144129 K: 192 MaxLP: -426322.5029144129
Step: 25 Time: -1536562205.751694 LP: -426066.9579232117 K: 192 MaxLP: -426066.9579232117
Step: 30 Time: -1536562201.458952 LP: -426066.9579232117 K: 192 MaxLP: -426066.9579232117
Step: 35 Time: -1536562197.1538298 LP: -425407.6212799563 K: 192 MaxLP: -425407.6212799563
Step: 40 Time: -1536562192.878116 LP: -424430.57098295185 K: 193 MaxLP: -424430.57098295185
Step: 45 Time: -1536562188.5288758 LP: -424341.983594564 K: 193 MaxLP: -424341.98359456396
Step: 50 Time: -1536562184.141099 LP: -424341.983594564 K: 193 MaxLP: -424341.9835945639